# Chapter 5 · Report

*This notebook turns Chapter 4 visuals into shareable outputs: per-cycle metrics, clean figures, and a compact HTML+CSV report.*

**For Chapter 5: Report**

(from “Biomechanics Data in Python: A Beginner’s Guide” by Hossein Mokhtarzadeh, PhD)

Visit PoseIQ.com for tools and demos.

Follow and learn more:

- Udemy courses: Hossein Mokhtarzadeh
- Amazon book: AI Mastery Series
- LinkedIn: PoseIQ™
- Demos (free and paid): PoseIQ Demos

## What you will do
- Detect stance from force signals using the same guards as Chapter 4
- Resample to cycle-normalized curves (0–100%)
- Compute per-cycle metrics that people actually use
- Save time plots and overlays
- Export a small report folder (CSV + HTML)

> Assumes `c3d` is already in memory. If not, use the setup cell below.

## Optional setup (run only if `c3d` is missing)

In [ ]:
# One-time setup (only if you did not run Chapter 2 here)
# If `c3d` is already in your session, you can skip this cell.
%pip -q install ezc3d pandas matplotlib ipywidgets

# Pull helper notebooks from earlier chapters if needed
!wget -q https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter1Input.py -O Chapter1Input.py
!wget -q https://raw.githubusercontent.com/hmok/BiomechPythonAI_Guide/main/notebooks/Chapter2Input.py -O Chapter2Input.py
%run Chapter1Input.py
%run Chapter2Input.py

# Quick check
try:
    _ = c3d["data"]["points"]
    print("c3d is loaded.")
except Exception as e:
    print("c3d was not found.", e)


## Helpers (shape-safe and label-aware, same as Chapter 4)

In [ ]:
import os, io, base64, json, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

P = c3d["parameters"]

def pval(g,k, default=None):
    try:
        return P[g][k]["value"]
    except Exception:
        return default

def get_rates():
    pt_rate = float(pval("POINT","RATE",[np.nan])[0]) if pval("POINT","RATE",None) is not None else np.nan
    an_rate = float(pval("ANALOG","RATE",[np.nan])[0]) if pval("ANALOG","RATE",None) is not None else np.nan
    return pt_rate, an_rate

def reshape_analogs():
    raw = c3d["data"].get("analogs", None)
    if raw is None:
        return np.zeros((0,0))
    A = np.array(raw)
    if A.ndim == 2:
        return A
    if A.ndim == 3:
        if A.shape[0] < 16 and A.shape[1] > A.shape[0]:
            return np.moveaxis(A, 0, -1).reshape(A.shape[1], -1)
        return A.reshape(A.shape[0], -1)
    return A.reshape(A.shape[0], -1)

def get_labels():
    point_labels = pval("POINT","LABELS", []) or []
    analog_labels = pval("ANALOG","LABELS", []) or []
    analog_units  = pval("ANALOG","UNITS",  []) or []
    return [str(x) for x in point_labels], [str(x) for x in analog_labels], [str(x) for x in analog_units]

def soft_detrend(y):
    return y - np.nanmedian(y)

def ensure_positive_down(y):
    pos_peak = np.nanpercentile(np.maximum(y, 0), 99)
    neg_peak = np.nanpercentile(np.maximum(-y, 0), 99)
    return y if pos_peak >= neg_peak else -y

def pick_fz_or_mag(an, labels):
    fz_idx = [i for i,s in enumerate(labels) if "fz" in s.lower() or "vf" in s.lower()]
    for i in fz_idx:
        y = an[i].astype(float)
        y = ensure_positive_down(soft_detrend(y))
        if np.nanmax(np.abs(y)) > 1e-6:
            return y, "Fz", i
    fx = next((i for i,s in enumerate(labels) if "fx" in s.lower()), None)
    fy = next((i for i,s in enumerate(labels) if "fy" in s.lower()), None)
    fz = next((i for i,s in enumerate(labels) if "fz" in s.lower()), None)
    if fx is not None and fy is not None and fz is not None:
        mag = np.sqrt(an[fx]**2 + an[fy]**2 + an[fz]**2).astype(float)
        return soft_detrend(mag), "GRF magnitude", (fx,fy,fz)
    return None, None, None

def clean_contact_bool(x, min_len_samples):
    x = x.astype(bool)
    on  = np.where(np.diff(x.astype(int)) == 1)[0] + 1
    off = np.where(np.diff(x.astype(int)) == -1)[0] + 1
    if x[0]:  on  = np.r_[0, on]
    if x[-1]: off = np.r_[off, len(x)]
    y = x.copy()
    for s,e in zip(on,off):
        if e - s < min_len_samples:
            y[s:e] = False
    return y

def detect_stance_pairs(sig, an_rate, thr_frac=0.05, thr_abs_N=20.0, min_stance_s=0.15):
    peak_est = np.nanpercentile(np.abs(sig), 98)
    thr = max(thr_frac * peak_est, thr_abs_N)
    contact = sig > thr
    contact = clean_contact_bool(contact, int(min_stance_s * an_rate))
    starts = np.where(np.diff(contact.astype(int)) == 1)[0] + 1
    ends   = np.where(np.diff(contact.astype(int)) == -1)[0] + 1
    if contact[0] and (len(ends) == len(starts) + 1): starts = np.r_[0, starts]
    if contact[-1] and (len(starts) == len(ends) + 1): ends = np.r_[ends, len(contact)]
    return [(s,e) for s,e in zip(starts,ends) if e > s]

def resample_cycle(y, s, e, n=101):
    if e - s < 3:
        return np.full(n, np.nan)
    xs = np.linspace(s, e - 1, e - s)
    xt = np.linspace(s, e - 1, n)
    return np.interp(xt, xs, y[s:e])


## Metrics (per-cycle, beginner friendly)
Set `MASS_KG` to get body-weight normalization.

In [ ]:
def per_cycle_metrics(sig, t, pairs, mass_kg=None):
    rows = []
    g = 9.81
    for k,(s,e) in enumerate(pairs):
        y = sig[s:e].astype(float)
        tt = t[s:e].astype(float)
        if len(y) < 3: 
            continue
        peakN = float(np.nanmax(y))
        target = 0.9 * peakN
        rise_idx = np.argmax(y > max(1e-9, target))
        rise_idx = max(int(rise_idx), 1)
        lr = (y[rise_idx] - y[0]) / max(tt[rise_idx] - tt[0], 1e-6)
        contact_time = float(tt[-1] - tt[0])
        out = {
            "Cycle": k+1,
            "PeakN": peakN,
            "ContactTime_s": contact_time,
            "LoadingRate_N_per_s": lr,
        }
        if mass_kg and mass_kg > 0:
            out["Peak_BW"] = peakN / (mass_kg * g)
            out["LoadingRate_BW_per_s"] = lr / (mass_kg * g)
        rows.append(out)
    return pd.DataFrame(rows)

def cadence_from_pairs(pairs, an_rate):
    if len(pairs) < 2:
        return np.nan
    starts = np.array([s for s,_ in pairs], dtype=float)
    dur_s  = (starts[-1] - starts[0]) / an_rate
    steps  = len(starts)
    return 60.0 * steps / max(dur_s, 1e-6)


## Figures (time, cycles, bars, scatter)

In [ ]:
def _save_fig(path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.tight_layout()
    plt.savefig(path, dpi=150, bbox_inches="tight")
    plt.show()

def fig_time_with_events(t, sig, pairs, title, ylabel, out_path):
    plt.figure(figsize=(8,3))
    plt.plot(t, sig, label="Signal")
    for s,e in pairs:
        plt.axvspan(t[s], t[e-1], alpha=0.2, label="Stance" if s==pairs[0][0] else None)
    plt.xlabel("Time [s]"); plt.ylabel(ylabel); plt.title(title); plt.grid(True, alpha=0.3)
    if pairs: plt.legend()
    _save_fig(out_path)

def fig_cycles_overlay(sig, pairs, npts, title, ylabel, out_path):
    if not pairs:
        return None
    X = np.linspace(0,100,npts)
    C = np.array([resample_cycle(sig, s, e, npts) for s,e in pairs])
    meanC = np.nanmean(C, axis=0)
    plt.figure(figsize=(8,3))
    for i in range(C.shape[0]):
        plt.plot(X, C[i], alpha=0.35)
    plt.plot(X, meanC, linewidth=2, label="Mean")
    plt.xlabel("Gait cycle [%]"); plt.ylabel(ylabel); plt.title(title); plt.grid(True, alpha=0.3); plt.legend()
    _save_fig(out_path)
    return C, meanC

def fig_bar_peaks(df, out_path):
    if df is None or df.empty:
        return
    mean_peak = df["PeakN"].mean() if "PeakN" in df else np.nan
    plt.figure(figsize=(4,3))
    plt.bar(["All cycles"], [mean_peak])
    plt.ylabel("Peak [N]")
    plt.title("Peak GRF average")
    _save_fig(out_path)

def fig_scatter_peaks(df, out_path):
    if df is None or df.empty:
        return
    plt.figure(figsize=(5,3))
    plt.scatter(np.arange(len(df))+1, df["PeakN"])
    plt.xlabel("Cycle #"); plt.ylabel("Peak [N]"); plt.title("Per cycle peaks")
    plt.grid(True, alpha=0.3)
    _save_fig(out_path)


## Report writer (CSV + HTML with embedded images)

In [ ]:
def _img_to_base64(path):
    with open(path, "rb") as f:
        import base64
        return "data:image/png;base64," + base64.b64encode(f.read()).decode("ascii")

def write_report(folder, meta, df_metrics, images):
    os.makedirs(folder, exist_ok=True)
    csv_path = os.path.join(folder, "metrics.csv")
    df_metrics.to_csv(csv_path, index=False)

    html_path = os.path.join(folder, "report.html")
    with open(html_path, "w") as f:
        f.write("<h1>Biomechanics Report</h1>")
        f.write("<h3>Summary</h3>")
        f.write("<pre>"+json.dumps(meta, indent=2)+"</pre>")
        f.write("<h3>Metrics</h3>")
        f.write(df_metrics.to_html(index=False))
        for title, p in images:
            if p and os.path.exists(p):
                f.write(f"<h3>{title}</h3>")
                f.write(f'<img src="{_img_to_base64(p)}" style="max-width:900px;">')
    return {"csv": csv_path, "html": html_path}


## One-touch run
Set `MASS_KG` if you want body-weight normalization.

In [ ]:
# Config
OUTPUT = "report_outputs"
CYCLE_NPTS = 101
MASS_KG = None  # e.g., 75.0 for BW normalization

# Data
pt_rate, an_rate = get_rates()
an = reshape_analogs()
point_labels, analog_labels, analog_units = get_labels()
sig, sig_name, src = pick_fz_or_mag(an, analog_labels)

if sig is None or an.shape[1] == 0 or np.isnan(an_rate):
    print("Could not build a reporting signal. Check analog labels and ANALOG.RATE.")
else:
    t_an = np.arange(an.shape[1]) / an_rate
    pairs = detect_stance_pairs(sig, an_rate, thr_frac=0.05, thr_abs_N=20.0, min_stance_s=0.15)
    print(f"Detected {len(pairs)} stance phases using {sig_name}")

    # Metrics
    df = per_cycle_metrics(sig, t_an, pairs, mass_kg=MASS_KG)
    cad = cadence_from_pairs(pairs, an_rate)
    meta = {
        "Signal": sig_name,
        "AnalogRate_Hz": float(an_rate) if not np.isnan(an_rate) else None,
        "Cycles": int(len(pairs)),
        "Cadence_spm": None if (cad is None or np.isnan(cad)) else float(cad),
        "BW_norm": bool(MASS_KG is not None),
    }

    # Figures
    fig1 = os.path.join(OUTPUT, "time_with_stance.png")
    fig_time_with_events(t_an, sig, pairs, f"{sig_name} over time", f"{sig_name}", fig1)

    cyc = fig_cycles_overlay(sig, pairs, CYCLE_NPTS, f"{sig_name} cycles", f"{sig_name}", os.path.join(OUTPUT, "cycles_overlay.png"))
    fig_bar_peaks(df, os.path.join(OUTPUT, "peaks_bar.png"))
    fig_scatter_peaks(df, os.path.join(OUTPUT, "peaks_scatter.png"))

    # Bundle
    imgs = [
        ("Time plot with stance", fig1),
        ("Cycle overlay", os.path.join(OUTPUT, "cycles_overlay.png")),
        ("Peak bar", os.path.join(OUTPUT, "peaks_bar.png")),
        ("Peak scatter", os.path.join(OUTPUT, "peaks_scatter.png")),
    ]
    out_paths = write_report(OUTPUT, meta, df, imgs)

    print("Saved:")
    print(" CSV:", out_paths["csv"])
    print(" HTML:", out_paths["html"])
    if df.empty:
        print("Note: no cycles with metrics. Adjust threshold or units if needed.")


## Optional: Left vs Right symmetry (skips cleanly if channels missing)

In [ ]:
labels_lower = [s.lower() for s in analog_labels]
try:
    fz1 = an[labels_lower.index(next(s for s in labels_lower if "fz1" in s))]
    fz2 = an[labels_lower.index(next(s for s in labels_lower if "fz2" in s))]
    fz1 = ensure_positive_down(soft_detrend(fz1))
    fz2 = ensure_positive_down(soft_detrend(fz2))

    pairs_L = detect_stance_pairs(fz1, an_rate)
    pairs_R = detect_stance_pairs(fz2, an_rate)

    CYCLE_NPTS = 101
    C_L = np.array([resample_cycle(fz1, s, e, CYCLE_NPTS) for s,e in pairs_L]) if pairs_L else np.empty((0, CYCLE_NPTS))
    C_R = np.array([resample_cycle(fz2, s, e, CYCLE_NPTS) for s,e in pairs_R]) if pairs_R else np.empty((0, CYCLE_NPTS))

    if C_L.size and C_R.size:
        X = np.linspace(0,100,CYCLE_NPTS)
        Lm = np.nanmean(C_L, axis=0); Rm = np.nanmean(C_R, axis=0)
        plt.figure(figsize=(8,3))
        plt.plot(X, Lm, label="Left"); plt.plot(X, Rm, label="Right")
        plt.xlabel("Gait cycle [%]"); plt.ylabel("Fz"); plt.title("Symmetry overlay"); plt.grid(True, alpha=0.3); plt.legend()
        os.makedirs(OUTPUT, exist_ok=True)
        plt.tight_layout(); plt.savefig(os.path.join(OUTPUT, "symmetry_overlay.png"), dpi=150, bbox_inches="tight"); plt.show()
        aL = np.trapz(Lm, x=X); aR = np.trapz(Rm, x=X)
        if (aL + aR) != 0:
            sym_idx = 100.0 * (aL - aR) / ((aL + aR) / 2.0)
            print(f"Symmetry index [%]: {sym_idx:.2f}")
    else:
        print("Symmetry skipped, missing L or R cycles.")
except StopIteration:
    print("Did not find explicit FZ1 or FZ2 labels, symmetry skipped.")
